# Genotation primers design example

Note: this notebooks if for primer-specific features. For general features use explore_mouse example
WARNING: primers features are under heavy refactoring, may not work properly


In [1]:
import pprint
from pathlib import *
from pycomfort.files import *

In [2]:
import sys

base = Path("..")
local = (base / "genotations").resolve()
if local.exists():
    sys.path.insert(0, Path("..").absolute().as_posix())
    sys.path.insert(0, local)
    print(sys.path)
else:
    base = Path(".")
%load_ext autoreload
%autoreload 2

[PosixPath('/home/antonkulaga/genotations/genotations'), '/home/antonkulaga/genotations/examples/..', '/home/antonkulaga/genotations/examples', '/home/antonkulaga/micromamba/envs/genotations/lib/python310.zip', '/home/antonkulaga/micromamba/envs/genotations/lib/python3.10', '/home/antonkulaga/micromamba/envs/genotations/lib/python3.10/lib-dynload', '', '/home/antonkulaga/micromamba/envs/genotations/lib/python3.10/site-packages']


In [3]:
examples = base / "examples"
data = examples / "data"
tprint(data)

data
	PRJNA543661_transcripts.parquet


In [126]:
import polars as pl
from genotations import genomes
from genotations.genomes import Annotations
from genotations import *
from genotations import ensembl
from genotations.quantification import *
from genotations.genomes import *
from genotations.primers import suggest_primers_for_transcript_by_exons, PrimerResults

In [5]:
pl.Config.set_tbl_width_chars(10000)
pl.Config.set_fmt_str_lengths(1000)
pl.Config.set_tbl_rows(20)

polars.cfg.Config

In [6]:
human = ensembl.human
mouse = ensembl.mouse
mouse

In [213]:
def print_primer_results(primer_results: PrimerResults, cleaned: bool = True):
    results =primer_results.get_cleaned_results() if cleaned else primer_results.results
    for r in results:
        print(r.PRIMER_LEFT_SEQUENCE, r.PRIMER_RIGHT_SEQUENCE, r.PRIMER_LEFT_TM, r.PRIMER_RIGHT_TM, r.PRIMER_PAIR_PRODUCT_SIZE)

## Traf3

In [21]:
traf_3 = mouse.annotations.with_gene_name_in("Traf3").exons().with_sequences(mouse.genome)
traf_3.annotations_df

seqname,source,feature,start,end,score,strand,frame,attribute,gene,gene_name,transcript_biotype,transcript,transcript_name,exon,exon_number,transcript_exon,coordinates,sequence
str,str,str,u64,u64,str,cat,str,str,str,str,str,str,str,str,u64,str,list[str],str
"""12""","""ensembl_havana""","""exon""",111132804,111133036,""".""","""+""",""".""","""gene_id ""ENSMUSG00000021277""; gene_version ""17""; transcript_id ""ENSMUST00000117269""; transcript_version ""8""; exon_number ""1""; gene_name ""Traf3""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Traf3-203""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS36563""; exon_id ""ENSMUSE00000712457""; exon_version ""2""; tag ""basic""; transcript_support_level ""1 (assigned to previous version 7)"";""","""ENSMUSG00000021277""","""Traf3""","""protein_coding""","""ENSMUST00000117269""","""Traf3-203""","""ENSMUSE00000712457""",1,"""Traf3-203_1""","[""12"", ""+"", ... ""111133036""]","""agcgcgcgAGAGGAAGTGCCAGCGCGAGGGTGCGTGAGGCGGCGCGGCCGGCGGCCGCCGCGTGCGCGAGCCGGGTTGCAGCCCAGCAGGGACTTTCCAGCcggcggcggcggcggcggcCGCCGGCCCTTCCCCGCCCCCCGACATGGGGCTGCCCGGGGAGCTGGACGCTGCAGAAGGCGGCGGAGGATGCGCGCGGCGCCCCTGAgccggccgaacgggcgggcctcggg"""
"""12""","""ensembl_havana""","""exon""",111203996,111204254,""".""","""+""",""".""","""gene_id ""ENSMUSG00000021277""; gene_version ""17""; transcript_id ""ENSMUST00000117269""; transcript_version ""8""; exon_number ""2""; gene_name ""Traf3""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Traf3-203""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS36563""; exon_id ""ENSMUSE00000311868""; exon_version ""2""; tag ""basic""; transcript_support_level ""1 (assigned to previous version 7)"";""","""ENSMUSG00000021277""","""Traf3""","""protein_coding""","""ENSMUST00000117269""","""Traf3-203""","""ENSMUSE00000311868""",2,"""Traf3-203_2""","[""12"", ""+"", ... ""111204254""]","""AATTTCAGTTTCCTAAGATGGAGTCAAGCAAAAAGATGGATGCTGCTGGCACACTGCAGCCTAACCCACCCCTAAAGCTGCAGCCTGATCGCGGCGCAGGGTCCGTGCTCGTGCCGGAGCAAGGAGGCTACAAGGAGAAGTTTGTGAAGACGGTGGAAGACAAGTACAAGTGCGAGAAGTGCCGCCTGGTGCTGTGCAACCCGAAGCAGACGGAGTGTGGCCACCGGTTCTGCGAGAGCTGCATGGCCGCCCTGCTGAG"""
"""12""","""ensembl_havana""","""exon""",111205463,111205514,""".""","""+""",""".""","""gene_id ""ENSMUSG00000021277""; gene_version ""17""; transcript_id ""ENSMUST00000117269""; transcript_version ""8""; exon_number ""3""; gene_name ""Traf3""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Traf3-203""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS36563""; exon_id ""ENSMUSE00000116300""; exon_version ""2""; tag ""basic""; transcript_support_level ""1 (assigned to previous version 7)"";""","""ENSMUSG00000021277""","""Traf3""","""protein_coding""","""ENSMUST00000117269""","""Traf3-203""","""ENSMUSE00000116300""",3,"""Traf3-203_3""","[""12"", ""+"", ... ""111205514""]","""CTCCTCCAGTCCAAAATGCACAGCGTGCCAAGAAAGCATCATCAAAGACAAG"""
"""12""","""ensembl_havana""","""exon""",111208877,111208981,""".""","""+""",""".""","""gene_id ""ENSMUSG00000021277""; gene_version ""17""; transcript_id ""ENSMUST00000117269""; transcript_version ""8""; exon_number ""4""; gene_name ""Traf3""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Traf3-203""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS36563""; exon_id ""ENSMUSE00000116306""; exon_version ""2""; tag ""basic""; transcript_support_level ""1 (assigned to previous version 7)"";""","""ENSMUSG00000021277""","""Traf3""","""protein_coding""","""ENSMUST00000117269""","""Traf3-203""","""ENSMUSE00000116306""",4,"""Traf3-203_4""","[""12"", ""+"", ... ""111208981""]","""GTGTTTAAGGATAATTGCTGCAAGAGAGAGATTCTGGCCCTTCAGGTCTACTGTCGGAATGAAGGCAGAGGTTGTGCGGAGCAGCTGAC

In [10]:
proper_traf_3 = traf_3.with_transcript_name_in("Traf3-201", "Traf3-204")
proper_traf_3.annotations_df

In [93]:
from genotations.intersections import *
traf_3_201_intervals =  TranscriptIntersection.extract_intervals(traf_3.with_transcript_name_in("Traf3-201"))

In [43]:
intervals_traf_3_201_204 = TranscriptIntersection.find_deepest_intersection(TranscriptIntersection.extract_intervals(proper_traf_3).to_list())
intervals_traf_3_201_204

In [241]:
primers_traf_3_201 = suggest_primers_for_transcript_by_exons(traf_3, "Traf3-201", 1 ,2)
print_primer_results(primers_traf_3_201)

CTGGACGCTGCAGAAGGC GGAGGGAGAAGCTTCCACAC 61.12362043794275 60.03591511653627 141
GCTGGACGCTGCAGAAGG GGAGGGAGAAGCTTCCACAC 61.12362043794275 60.03591511653627 142
CTGGACGCTGCAGAAGGC CGTGCCAGCCTTATCCTTCA 61.12362043794275 60.10770297561146 109
GCTGGACGCTGCAGAAGG CGTGCCAGCCTTATCCTTCA 61.12362043794275 60.10770297561146 110
CTGGACGCTGCAGAAGGC AAGGGAGGGAGAAGCTTCCA 61.12362043794275 60.178861489964106 144


In [243]:
primers_traf_3_204 = suggest_primers_for_transcript_by_exons(traf_3, "Traf3-204", 1 ,3)
print_primer_results(primers_traf_3_204)

CTGGACGCTGCAGAAGGC CGTGCCAGCCTTATCCTTCA 61.12362043794275 60.10770297561146 218
GCTGGACGCTGCAGAAGG CGTGCCAGCCTTATCCTTCA 61.12362043794275 60.10770297561146 219
CTGGACGCTGCAGAAGGC CCGTGCCAGCCTTATCCTTC 61.12362043794275 60.81561548754888 219
GCTGGACGCTGCAGAAGG CCGTGCCAGCCTTATCCTTC 61.12362043794275 60.81561548754888 220


In [281]:
print_primer_results(suggest_primers_for_transcript_by_exons(traf_3, "Traf3-201", 3,4, opt_t=61))

CGGAGCAAGGAGGCTACAAG TGATGCTTTCTTGGCACGCT 60.46098669932809 60.890542431921176 186
TAAAGCTGCAGCCTGATCGC CGCTGTGCATTTTGGACTGG 61.09177110573438 60.38762174135786 212
CGGAGCAAGGAGGCTACAAG ATGCTTTCTTGGCACGCTGT 60.46098669932809 61.17471180489525 184


In [275]:
print_primer_results(suggest_primers_for_transcript_by_exons(traf_3, "Traf3-204", 4 ,5, opt_t=61))

CGGAGCAAGGAGGCTACAAG TGATGCTTTCTTGGCACGCT 60.46098669932809 60.890542431921176 186
TAAAGCTGCAGCCTGATCGC CGCTGTGCATTTTGGACTGG 61.09177110573438 60.38762174135786 212
CGGAGCAAGGAGGCTACAAG ATGCTTTCTTGGCACGCTGT 60.46098669932809 61.17471180489525 184


## NF2

In [478]:
nf2 = mouse.annotations.with_gene_name_in("Nf2").exons().with_sequences(mouse.genome, strand=Strand.Undefined)
nf2.annotations_df

There are (126, 17) annotations, loading sequences can take quite a while!


seqname,source,feature,start,end,score,strand,frame,attribute,gene,gene_name,transcript_biotype,transcript,transcript_name,exon,exon_number,transcript_exon,coordinates,sequence
str,str,str,u64,u64,str,cat,str,str,str,str,str,str,str,str,u64,str,list[str],str
"""11""","""ensembl_havana""","""exon""",4798872,4799536,""".""","""-""",""".""","""gene_id ""ENSMUSG00000009073""; gene_version ""17""; transcript_id ""ENSMUST00000109910""; transcript_version ""9""; exon_number ""1""; gene_name ""Nf2""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Nf2-205""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS24391""; exon_id ""ENSMUSE00000794729""; exon_version ""2""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""1 (assigned to previous version 8)"";""","""ENSMUSG00000009073""","""Nf2""","""protein_coding""","""ENSMUST00000109910""","""Nf2-205""","""ENSMUSE00000794729""",1,"""Nf2-205_1""","[""11"", ""-"", ... ""4799536""]","""CTCGCAGTTGAACTCCATCTCGGCGTCCATGGTGACGATCCGCACCGTGAATGTCTTGGGCTGCTTCCTCTTGAGTGAGCTGAAGCTCATGCGAGAAGCGATGGCTCCGGCCATGGCGCGAGGTACCGGCCTTGGATCCCCTCACCCCTAGGCCCGCGTTCTGGGTCTCTTAGCCCCCCTGGAGTAGCTGCACAGGCCTCAGGGCCACCATGGTGGCCGGCTGGCCGGCCCGAGACAGACCTAGGCTGCCTGGGTCCCCAGCGGCCAGCATGGGCCGCGAGGACCCGGCTGTGGGTGCGGAAAGATGAGTTAAGGGACAGTCTCTGACGACgggggggtcggggggAGATGCCGGCAAAAGCACGCGCAGCCCCGAGCTTGGAGCTTCGGGCTGGGGACGCCAGGGACGCCTTTGGACAGAAATCCTAGACGGCTGCGGCGCCGACGCCGGGCCTCTCAGCGACCCCCGAGGGTCGCGGGGCCGCGCAGTAGTTGGGAGCGCACACGCGCGTGCGTGTCTGTCACCAGCTCCGCTCCGCGCGCCCTCGGGAAGCGTCCTCTACCAACCCCCACCTCAGCGACCTGCCGGATCCCCGCCCCGCCCCGCCCTCCGGACTCCCACTACGCTACTCAGCGACTCACCCTAACACTTTTTTCTCTAAGAT"""
"""11""","""ensembl_havana""","""exon""",4770368,4770493,""".""","""-""",""".""","""gene_id ""ENSMUSG00000009073""; gene_version ""17""; transcript_id ""ENSMUST00000109910""; transcript_version ""9""; exon_number ""2""; gene_name ""Nf2""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Nf2-205""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS24391""; exon_id ""ENSMUSE00001268327""; exon_version ""2""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""1 (assigned to previous version 8)"";""","""ENSMUSG00000009073""","""Nf2""","""protein_coding""","""ENSMUST00000109910""","""Nf2-205""","""ENSMUSE00001268327""",2,"""Nf2-205_2""","[""11"", ""-"", ... ""4770493""]","""CTTCTTGTCCATTTTGAGCCAGGCCACCGTGTCCTTGATTGTATACTGCAGTCCAAAGAACCAGGTTTCCCGAAGCCCCAGTGTCCGGCACACCAAATCAAACAGGTCCTTCCCCTTCCATTTCAT"""
"""11""","""ensembl_havana""","""exon""",4768505,4768627,""".""","""-""",""".""","""gene_id ""ENSMUSG00000009073""; gene_version ""17""; transcript_id ""ENSMUST00000109910""; transcript_version ""9""; exon_number ""3""; gene_name ""Nf2""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Nf2-205""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS24391""; exon_id ""ENSMUSE00001265598""; exon_version ""2""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""1 (assigned to previous version 8)"";""","""ENSMUSG00000009073""","""Nf2""","""protein_coding""","""ENSMUST00000109910""","""Nf2-205""","""ENSMUSE00001265598""",3,"""Nf2-205_3""","[""11"", ""-"", ... ""4768627""]","""CTGTAAGAAAAATAAGTGTTGCGTGATCTCTTGAACTAGCTCCTCCTCAGCATTTTCAGGATAAAATTTGGCCAGGAAGTGAAAGGTAACTGGTTCTTCCTTCGAAACATCATGATCCAACAC"""
"""11""","""ensembl_havana""","""exon""",4766082,4766165,""".""","""-""",""".""","""gene_id ""ENSMUSG00000009073""; gene_version ""17""; transcript_id ""ENSMUST00000109910""; transcript_version ""9""; exon_number ""4""; gene_name ""Nf2""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Nf2-205""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS24391""; exon_id ""ENS

In [481]:
print_primer_results(suggest_primers_for_transcript_by_exons(nf2, "Nf2-205", 2, 3, opt_t=61))

In [482]:
print_primer_results(suggest_primers_for_transcript_by_exons(nf2, "Nf2-202", 2, 3, opt_t=61))

In [484]:
print_primer_results(suggest_primers_for_transcript_by_exons(nf2, "Nf2-205", 13, 14, opt_t=62))

GCTTCTGCTTGGCTCTTCGC ATTCCACCACCACTGCCTCC 62.250841021206554 62.14229191265247 190
GCTTCTGCTTGGCTCTTCGC CCACCACTGCCTCCTGACAT 62.250841021206554 61.845626943902175 184


In [344]:
primers_Nf2_202 = suggest_primers_for_transcript_by_exons(nf2, "Nf2-202", 13, 14, opt_t=62)
print_primer_results(primers_Nf2_202)

GCGAAGAGCCAAGCAGAAGC GGAGGCAGTGGTGGTGGAAT 62.250841021206554 62.14229191265247 84
GCGAAGAGCCAAGCAGAAGC ATGTCAGGAGGCAGTGGTGG 62.250841021206554 61.845626943902175 90
GCGAAGAGCCAAGCAGAAGC GCTCGGTATGTCAGGAGGCA 62.250841021206554 61.67667823857579 97
AGCCAGAGAAGCAGAGCGAA GGAGGCAGTGGTGGTGGAAT 61.5395601118949 62.14229191265247 99
AAGCCAGAGAAGCAGAGCGA GGAGGCAGTGGTGGTGGAAT 61.5395601118949 62.14229191265247 100


# Dtx2

In [153]:
dtx2= mouse.annotations.with_gene_name_in("Dtx2").exons().with_sequences(mouse.genome)
dtx2.annotations_df

seqname,source,feature,start,end,score,strand,frame,attribute,gene,gene_name,transcript_biotype,transcript,transcript_name,exon,exon_number,transcript_exon,coordinates,sequence
str,str,str,u64,u64,str,cat,str,str,str,str,str,str,str,str,u64,str,list[str],str
"""5""","""ensembl_havana""","""exon""",136023654,136023820,""".""","""+""",""".""","""gene_id ""ENSMUSG00000004947""; gene_version ""16""; transcript_id ""ENSMUST00000111142""; transcript_version ""9""; exon_number ""1""; gene_name ""Dtx2""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Dtx2-202""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS57389""; exon_id ""ENSMUSE00000836594""; exon_version ""2""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""1 (assigned to previous version 8)"";""","""ENSMUSG00000004947""","""Dtx2""","""protein_coding""","""ENSMUST00000111142""","""Dtx2-202""","""ENSMUSE00000836594""",1,"""Dtx2-202_1""","[""5"", ""+"", ... ""136023820""]","""TGGACATGCGCAGCGCGGTGCTTGGAGGCTGCGACGCCGAGTTTCACTCCGGCTGCCTCCGGCTCGGAGTCGGAGACATACTCGAGGGTTCTTCCCAGGCCGAGCCGGAGCTGAGCCCGGAGGTCGGCGGTCCTCTCAACCTGTTCCTTCTGCTTCCCTCAGACTCG"""
"""5""","""ensembl_havana""","""exon""",136025144,136025306,""".""","""+""",""".""","""gene_id ""ENSMUSG00000004947""; gene_version ""16""; transcript_id ""ENSMUST00000111142""; transcript_version ""9""; exon_number ""2""; gene_name ""Dtx2""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Dtx2-202""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS57389""; exon_id ""ENSMUSE00000191750""; exon_version ""2""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""1 (assigned to previous version 8)"";""","""ENSMUSG00000004947""","""Dtx2""","""protein_coding""","""ENSMUST00000111142""","""Dtx2-202""","""ENSMUSE00000191750""",2,"""Dtx2-202_2""","[""5"", ""+"", ... ""136025306""]","""GTACCTCGGAATCGGTGTGGCTGAGTCAGCAAGGCCTCAGATTTGCCGGGTTCTTGTTTGCTGTTCCTATCAAGATGGGAGCTCAGACAAGTTATTGCTTCTGAGGATCTGGTGCTTTCATCAAGAAGGGACAGTTTGGTGCCAGCTCTTCAGTGAAGAGGAG"""
"""5""","""ensembl_havana""","""exon""",136038868,136039210,""".""","""+""",""".""","""gene_id ""ENSMUSG00000004947""; gene_version ""16""; transcript_id ""ENSMUST00000111142""; transcript_version ""9""; exon_number ""3""; gene_name ""Dtx2""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Dtx2-202""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS57389""; exon_id ""ENSMUSE00000191757""; exon_version ""2""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""1 (assigned to previous version 8)"";""","""ENSMUSG00000004947""","""Dtx2""","""protein_coding""","""ENSMUST00000111142""","""Dtx2-202""","""ENSMUSE00000191757""",3,"""Dtx2-202_3""","[""5"", ""+"", ... ""136039210""]","""GTCCCCCAGAGGTGCTGGGCAATGAACCTGAACCTGCAACCCAGCGTGGTGTGAAGTCTTCAGAGAAAGCAGAAGATGGCCATGGCCCCAAGCTCGTCTCTGCCGCAGGTGTACCCTAGCCACGTGGTGGTGGCTGTGTGGGAATGGCAGGATGGGCTTGGCATTTGGCACCCCTACAGTGCCACTGTCTGCTCCTTCATTGAGCAGCATTTTGTCCGGCAGAGGGGCCAGCATTTTGGACTGGGAAGCCTGGCCCACAGCATCCCCTTAGGCCAAGCTGACCCCTCACTGGCCCCTTACATCATCGACCTCCCCAGCTGGACTCAGTTTCGCCAGAACACTG"""
"""5""","""ensembl_havana""","""exon""",136040865,136041501,""".""","""+""",""".""","""gene_id ""ENSMUSG00000004947""; gene_version ""16""; transcript_id ""ENSMUST00000111142""; transcript_version ""9""; exon_number ""4""; gene_name ""Dtx2""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Dtx2-202""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS57389""; exon_id ""ENSMUSE00000191748""; exon_version ""2""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""1 (assigned to previous version 8)"";""","""ENSMUSG00000004947""","""Dtx2""","""protein_coding""","""

In [165]:
primers_Dtx2_202 = suggest_primers_for_transcript_by_exons(dtx2, "Dtx2-202", 1 ,2, opt_t=60)
print_primer_results(primers_Dtx2_202)

CGGTCCTCTCAACCTGTTCC AAGAGCTGGCACCAAACTGT 60.03723245398322 60.10651921155221 190
CGGTCCTCTCAACCTGTTCC GGCACCAAACTGTCCCTTCT 60.03723245398322 60.17898958797463 183


In [166]:
primers_Dtx2_204 = suggest_primers_for_transcript_by_exons(dtx2, "Dtx2-204", 1 ,2, opt_t=60)
print_primer_results(primers_Dtx2_204)

CGGTCCTCTCAACCTGTTCC AAGAGCTGGCACCAAACTGT 60.03723245398322 60.10651921155221 190
CGGTCCTCTCAACCTGTTCC GGCACCAAACTGTCCCTTCT 60.03723245398322 60.17898958797463 183


In [409]:
print_primer_results(suggest_primers_for_transcript_by_exons(dtx2, "Dtx2-202", 8, 9, opt_t=63))

In [407]:
print_primer_results(suggest_primers_for_transcript_by_exons(dtx2, "Dtx2-204", 7,9, opt_t=64))

## Zswim8-201

In [410]:
zswim8 = mouse.annotations.with_gene_name_in("Zswim8").exons().with_sequences(mouse.genome)
zswim8.annotations_df

There are (150, 17) annotations, loading sequences can take quite a while!


seqname,source,feature,start,end,score,strand,frame,attribute,gene,gene_name,transcript_biotype,transcript,transcript_name,exon,exon_number,transcript_exon,coordinates,sequence
str,str,str,u64,u64,str,cat,str,str,str,str,str,str,str,str,u64,str,list[str],str
"""14""","""ensembl_havana""","""exon""",20757620,20758124,""".""","""+""",""".""","""gene_id ""ENSMUSG00000021819""; gene_version ""12""; transcript_id ""ENSMUST00000022358""; transcript_version ""9""; exon_number ""1""; gene_name ""Zswim8""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Zswim8-201""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS36820""; exon_id ""ENSMUSE00000978077""; exon_version ""2""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""1 (assigned to previous version 7)"";""","""ENSMUSG00000021819""","""Zswim8""","""protein_coding""","""ENSMUST00000022358""","""Zswim8-201""","""ENSMUSE00000978077""",1,"""Zswim8-201_1""","[""14"", ""+"", ... ""20758124""]","""AGCTGATGGGGGAGGCGGCGCCGCAGCCGCCTAGAGGCCCTGGCCGCCCAGCGCTTCGTCCCGGCCTCGAGTCTCGCAGGCTGGCCACAATCCCCGCTCGCACCTCGCTGTTGGGTGAGGCCCGGCCCGGTGTCCCTCCCAGGCCCGAGATCCTTgcccggcacggccgccctgagcgccccggccacccccagccccggctcgcccctcaggccccgggcctcccctcagtccccggccggcggcccaggccccggatccgcggggggagacccggccccggggggtgcgggccccATGGAGCTGATGTTCGCGGAGTGGGAGGACGGCGAGCGCTTCTCGTTTGAGGATTCCGACCGCTTTGAGGAGGATTCACTCTGTTCGTTCATTTCCGAGGCCGAGAGCCTTTGCCAGAACTGGCGAGGATGGCGCAAACAGTCAGCGGGGCCCAATTCCCCCACTGGAGGCGGTGGCGGAGGTGGCAGTGGCGGTACCAGAACGCGAG"""
"""14""","""ensembl_havana""","""exon""",20760705,20760858,""".""","""+""",""".""","""gene_id ""ENSMUSG00000021819""; gene_version ""12""; transcript_id ""ENSMUST00000022358""; transcript_version ""9""; exon_number ""2""; gene_name ""Zswim8""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Zswim8-201""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS36820""; exon_id ""ENSMUSE00001037343""; exon_version ""2""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""1 (assigned to previous version 7)"";""","""ENSMUSG00000021819""","""Zswim8""","""protein_coding""","""ENSMUST00000022358""","""Zswim8-201""","""ENSMUSE00001037343""",2,"""Zswim8-201_2""","[""14"", ""+"", ... ""20760858""]","""ATGGATTGGTAATCCCATTGGTGGAGCTGTCAGCAAAGCAGGTGGCATTTCACATCCCATTTGAAGTGGTAGAGAAAGTTTATCCTCCAGTGCCAGAACAACTCCAACTCCGAATTGCTTTTTGGAGCTTCCCTGAGAATGAAGAGGACATTCG"""
"""14""","""ensembl_havana""","""exon""",20761096,20761190,""".""","""+""",""".""","""gene_id ""ENSMUSG00000021819""; gene_version ""12""; transcript_id ""ENSMUST00000022358""; transcript_version ""9""; exon_number ""3""; gene_name ""Zswim8""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Zswim8-201""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS36820""; exon_id ""ENSMUSE00001043732""; exon_version ""2""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""1 (assigned to previous version 7)"";""","""ENSMUSG00000021819""","""Zswim8""","""protein_coding""","""ENSMUST00000022358""","""Zswim8-201""","""ENSMUSE00001043732""",3,"""Zswim8-201_3""","[""14"", ""+"", ... ""20761190""]","""TCTGTATTCATGCCTAGCCAATGGCAGTGCGGATGAGTTTCAGCGAGGGGATCAGCTGTTCCGAATGAGGGCTGTGAAAGACCCGCTGCAGATAG"""
"""14""","""ensembl_havana""","""exon""",20761526,20761698,""".""","""+""",""".""","""gene_id ""ENSMUSG00000021819""; gene_version ""12""; transcript_id ""ENSMUST00000022358""; transcript_version ""9""; exon_number ""4""; gene_name ""Zswim8""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""Zswim8-201""; transcript_source ""ensembl_havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS36820""; exon_id ""ENSMUSE00001056494""; exon_version ""2""; tag ""basic""; tag ""Ensembl_canonical""; transcript_suppor

In [232]:
primers_zswim8_201 = suggest_primers_for_transcript_by_exons(zswim8, "Zswim8-201", 2 ,4, opt_t=62)
print_primer_results(primers_zswim8_201)

TCCAGTGCCAGAACAACTCCA CTTGGGTGGGACCATTTGCG 61.59169064857434 61.87649736648012 214
GCCAGAACAACTCCAACTCCG CTTGGGTGGGACCATTTGCG 61.47438713861044 61.87649736648012 208


In [464]:
print_primer_results(suggest_primers_for_transcript_by_exons(zswim8, "Zswim8-214", 2,3, opt_t=61))

TGGCTCCCATCACAGTCCAT CACCCTGGATGGCTGGAAAC 60.915872158419916 60.9673804944149 111
ATGGCTCCCATCACAGTCCA CACCCTGGATGGCTGGAAAC 60.915872158419916 60.9673804944149 112
TCCATGGCTCCCATCACAGT CACCCTGGATGGCTGGAAAC 60.915872158419916 60.9673804944149 115
